In [1]:
import os
import sys
import json
import dspy

d:\ThirdSummer\Something\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
from dspy import ColBERTv2

llama = dspy.GROQ(model="llama3-8b-8192", api_key=os.environ["GROQ_API_KEY"])
mini = dspy.OpenAI(model="gpt-4o-mini", api_key=os.environ["OPENAI_API_KEY"])
colber_wiki = ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")

dspy.settings.configure(lm=mini, rm=colber_wiki)

In [30]:
from input_data.pollutes import pollution_issues

print("Pollution issues: ", pollution_issues)


Pollution issues:  ['Lead Poisoning', 'Fracking Fluids', 'Oil Spills', 'Nitrate Pollution', 'Plastic Microbeads', 'Chlorine Byproducts', 'Arsenic Contamination', 'Radioactive Waste', 'Mercury Bioaccumulation', 'Phosphate Runoff', 'Endocrine Disruptors', 'Acid Mine Drainage', 'Cyanobacterial Blooms', 'Landfill Leachate', 'Thermal Pollution', 'E. coli Outbreaks', 'Dioxin Contamination', 'Salinization', 'PCB Pollution', 'Groundwater Depletion', 'Petroleum Hydrocarbons', 'Antibiotic Resistance', 'Nutrient Overload', 'Sediment Pollution', 'PFAS Chemicals', 'Viral Pathogens', 'Organic Solvents', 'Livestock Effluents', 'Stormwater Runoff', 'Desalination Brine', 'Fluoride Excess', 'Cryptosporidium Outbreaks', 'Benzene Leaks', 'Microplastic Pollution', 'Perchlorate Contamination', 'Sulfate Runoff', 'Trihalomethanes (THMs)', 'Legionella Growth', 'Chromium Pollution', 'Nanoparticle Contamination', 'Asbestos Fibers', 'Cyanide Spills', 'Methane Seepage', 'Radium Leaching', 'Chloramine Byproducts', 

In [31]:
class GenerateData(dspy.Signature):
    """ Generate a simple trivia question from topic about pollution issues. """
    topic = dspy.InputField()
    question = dspy.OutputField(desc="Simple Trivia Question")
    answer = dspy.OutputField(desc="Answer to the trivia question 1 to 2 words")
    
import random
def temp_set():
    return dict(temperature=0.7 + 0.001 * random.uniform(-1, 1))

class GenerateTrivia(dspy.Module):
    def __init__(self):
        self.retreive = dspy.Retrieve(k=2)
    
    def forward(self, query):
        contexts = self.retreive(query).passages
        answers = []
        for context in contexts:
            output = dspy.ChainOfThought(GenerateData, **temp_set())(topic=context)
            answers.append({
                "context": context,
                "question": output.question,
                "answer": output.answer
            })
            
        return answers
    
    

In [32]:
trivia_gen = GenerateTrivia()
# generate output, save outputs and store in a dataframe
triva = []

for p in pollution_issues:
    output = trivia_gen(p)
    triva.append(output)
    

ReadTimeout: HTTPConnectionPool(host='20.102.90.50', port=2017): Read timed out. (read timeout=10)

In [1]:
import pandas as pd

df = []
for t in triva:
    df += t

df = pd.DataFrame(df)

df.to_csv("./input_data/pollution_trivia_gpt4omini.csv", index=False)


NameError: name 'triva' is not defined

In [20]:
df_llama = pd.read_csv("./input_data/pollution_trivia_llama.csv")
import json
df_ = []
# modify the dataframe to exclude the dict
for i, row in df_llama.iterrows():
    context = eval(row["context"])
    question = eval(row["question"])
    answer = eval(row["answer"])
    df_.append([context['context'], context['question'], context['answer']])
    df_.append([question['context'], question['question'], question['answer']])
    df_.append([answer['context'], answer['question'], answer['answer']])

In [22]:
df_ = pd.DataFrame(df_, columns=["context", "question", "answer"])
df_.to_csv("./input_data/pollution_trivia_llama_fix.csv", index=False)